# Measuring Personalization Performance Using Amplitude
In this exercise you will explore the conversion performance of your Personalize deployment in Retail Demo Store using Amplitude's analytics platform.

*Recommended Time: 30 minutes*

## Prerequisites

This module uses the Retail Demo Store's Recommendation service to run experiments across variations that depend on the personalization features of the Retail Demo Store, it is assumed that you have either completed the Personalization workshop or those resources have been pre-provisioned in your AWS environment. If you are unsure and attending an AWS managed event such as a workshop, check with your event lead.

This project also assumes that Amplitude has been configured during the deployment of the Retail Demo Store in your AWS account or managed event account. 

In order to deploy Amplitude, you will need access to an Amplitude workspace, and an Amplitude API key that allows you to send data to Amplitude.  We recommend setting up a separate Amplitude project for the workshop, so that you can delete the demo data that will be passed to Amplitude as part of the workshop when you are finished.

## Overview

Deploying personalizaiton can be challenging in two ways:

1. Deciding where to deploy personalized experiences is often done using best guesses
2. Measuring the conversion outcomes of personalized experiences once they are deployed can be difficult without proper tooling

Amplitude addresses these issues by providing a comprehensive real-time measurement platform that allows you to track user behavior in the form of real-time events before and after the deployment of Personalize. 

In this workshop, you will set up tracking for Amplitude events, analyze user behavior prior to peronalization being deployed, and then measure the effects of personalization on user behavior after Personalize is deployed in the Retail Demo Store.

## Set Up Amplitude

Amplitude works by collecting real-time data from user behavior in the Retail Demo Store website.  The Amplitude tracking code is included in the workshop for you, so user interactions with the application will be collected in Amplitude if you included your Amplitude API key during the deployment of the Retail Demo Store environment.

Because the Retail Demo Store is developed using the Vue framework, we can load Amplitude when the user loads a page by including the Amplitude tracking script using an import statement in `../src/web-ui/src/analytics/AnalyticsHandler.js`:

```
/* 
 * Centralized handling of all analytics calls for Pinpoint, Personalize 
 * (event tracker), and partner integrations.
 */
import Vue from 'vue';
import { Analytics as AmplifyAnalytics } from '@aws-amplify/analytics';
import Amplitude from 'amplitude-js'
```

Once the tracking script is loaded into a page, it can be used to fire events in response to user actions.  As an example, when a user clicks on a product on one of the Retail Demo Store pages, a ProductViewed event will be sent to Amplitude, using this code:

```
if (this.amplitudeEnabled()) {
    // Amplitude event
    var eventProperties = {
        productId: product.id,
        name: product.name,
        category: product.category,
        image: product.image,
        feature: feature,
        experimentCorrelationId: experimentCorrelationId,
        price: +product.price.toFixed(2)
    };
    Amplitude.getInstance().logEvent('ProductViewed', eventProperties);
}
```

These events are the same as events that are used to train the Personalize models used to offer users Personalize recommendations.  This is important since you will want the reports in Amplitude's analytics platform to reflect the same information that is being used to create Personalize models.  This will give you the most accurate way to measure user behavior before Personalize is deployed, and a consistent way to measure user behavior after Personalize is deployed in the Retail Demo Store site.

Retail Demo Store sends the following events to Personalize and Amplitude:

TODO: Add markdown for table of events from blog post here

## Measuring Conversion Before Personalization is Deployed

Because Amplitude reports rely on real user data, you will be using a data simulator to create enough events for you to be able to run a useful report in Amplitude.  The data simulator included in the Retail Demo Store will send the events shown above as though they were being generated by real users of the Retail Demo Store.  This will simulate common browsing behaviors at a sufficent volume to allow you to run a report that looks like one that might be used in your own production applications.

The first simulated set of data will represent the Retail Demo Store prior to the deployment of the three personalization features you deployed in the first workshop exercise.

Modify the code below to add in your Amplitude API key, and then run the code to send events to your Amplitude instance.

TODO: Modify configs below to accept a write key directly from the code rather than a config file


In [5]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../generators'))

import datetime
from datagenerator.rdscatalog import RDSCatalog
from datagenerator.rdsuserstate import RDSUserSelectionState
from datagenerator.users import UserPool
from datagenerator.sessions import Sessions
from datagenerator.output import OutputWriter


search_props_template = {
  'query': RDSUserSelectionState.user_search,
  'resultCount': RDSUserSelectionState.num_results,
  'reranked': False
}

search_props_feature_template = {
  'query': RDSUserSelectionState.user_search,
  'resultCount': RDSUserSelectionState.num_results,
  'reranked': True
}

prod_clicked_template = {
  'expand': RDSUserSelectionState.item_viewed_event_props
}

prod_clicked_feature_template = {
  'expand': RDSUserSelectionState.item_viewed_event_props,
  'feature': ['home_product_recs', 'product_detail_related']
}

prod_added_template = {
  'expand': RDSUserSelectionState.item_added_event_props,
}

checkout_start_template = {
  'expand': RDSUserSelectionState.cart_viewed_event_props,
}

order_completed_template = {
  'expand': RDSUserSelectionState.cart_viewed_event_props
}

catalog = RDSCatalog('../../src/products/src/products-service/data/products.yaml')

usage_funnels_pre_personalize = [
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductSearched', search_props_template),
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductsSearched', prod_clicked_template)
    ]
  }
]

date_time_now = datetime.datetime.now()

pool = UserPool('../../src/users/src/data/users.json')  # Load the known pool of users from the users file
print(f'Loaded {pool.size()} users...')
# Sends events to simulate application prior to personalization
sessions = Sessions(date_time_now - datetime.timedelta(days=60), date_time_now - datetime.timedelta(days=30), usage_funnels_pre_personalize, 100, pool)
writer = OutputWriter(sessions)
#writer.to_amplitude('amplitude_config.yaml', True)

FileNotFoundError: [Errno 2] No such file or directory: '../../src/users/src/data/users.json'

## Create Amplitude Conversion Reports

Once the event generator has completed, log in to your Amplitude console.

In the console, you will use Amplitude's Funnel Analysis report to generate a report that enables you to see the conversion rate of the Retail Demo Store main purchase funnel.  Amplitude allows you to build a funnel for any user flow in your applications.  For the Retail Demo Store, you will build a report that tracks all user sessions that perform the following actions in this order:

ProductViewed >
ProductAdded >
CheckoutStarted >
OrderCompleted

Note that there might be other steps in the middle of this flow, such as users viewing other products, adding and removing items from their cart, etc. but the Funnel report will ignore those events for the purpose of this report, since we are looking to see only conversions for users that have viewed a particular product and then purchased it through the minimum checkout flow steps for the Retail Demo Store.

![](images/amplitude/conversion-funnel-pre-pers.png)

Note that the purchase conversion rate for the main funnel in the Retail Demo Store is 23.8%!  Not bad, but would we see an improvement in conversion if users were to receive personalized recommendaitons while they are interacting with the site? 

TODO:  Add all screen shots from workshop blog


## Measuring the Conversion Effects of Personalization

In the first workshop, you deployed the following Personalization features in the Retail Demo Store:

1. Personalized search results re-ranking
2. Personalized product recommendations
3. Similar product recommendations

In order to measure how users convert after interacting with these features, we need to add a parameter to the event tracking sent to Amplitude, so that our funnel report can differentiate between organic product views (views that are not influenced via personalization features) and product views and purchases that were as a result of user interaction with the newly-added personalization features.

In the `AnalyticsHandler.js` file, you will see code that adds a parameter to the ProductViewed event:

```
if (this.amplitudeEnabled()) {
    // Amplitude event
    var eventProperties = {
        productId: product.id,
        name: product.name,
        category: product.category,
        image: product.image,
        feature: feature,
        experimentCorrelationId: experimentCorrelationId,
        price: +product.price.toFixed(2)
    };
    Amplitude.getInstance().logEvent('ProductViewed', eventProperties);
}
```

Note the `feature` property.  When the user clicks on a product on one of the three personalization features, this property will contain the name of the personalization feature the user clicked on.  

This property will allow us to create a funnel report in Amplitude that shows the conversion rate of products clicked by users as a result of viewing a Personalization feature.


In [ ]:
usage_funnels_post_personalize_features = [
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_feature_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template)
    ]
  },
  {
    'platform': 'web',
    'state': lambda user: RDSUserSelectionState(catalog, user),
    'templates': [
      ('ProductViewed', prod_clicked_template),
      ('ProductAdded', prod_added_template),
      ('CartViewed', prod_clicked_template),
      ('CheckoutStarted', checkout_start_template),
      ('OrderCompleted', order_completed_template)
    ]
  }
]

# Sends events to simulate application after personalization features are added
sessions = Sessions(date_time_now - datetime.timedelta(days=30), date_time_now, usage_funnels_post_personalize_features, 100, pool)
writer = OutputWriter(sessions)
writer.to_amplitude('amplitude_config.yaml', False)

## Create Amplitude Personalization Conversion Reports 

Once the event generator has completed, go back to the Amplitude console.

In the console, you will once again use Amplitude's Funnel Analysis report to generate a report that enables you to see the conversion rate of the Retail Demo Store main purchase funnel, this time after the Personalize features have been deployed.  We will still track the normal conversion funnel as before, but this time we are going to drill down into ProductViewed events that were a result of the three personalizaiton features we deployed earlier.

ProductViewed >
ProductAdded >
CheckoutStarted >
OrderCompleted

Instead of looking at all user paths through this funnel, we will look individually at the number of conversions that happen when for unique users sessions that start with an interaction with a personalization feature.  Under the ProductViewed event, select `where` and then the `feature` property.  This will show a dropdown window that allows you to specify which values for the feature will be selected for the report.  In this case, we will look for events that are tied to 'product_detail_related' and 'home_product_recs'.  These values correspond to the Related Products feature, and the Product Recommendations feature for users that land on the home page.

![](./images/amplitude/product-viewed-feature-select.png)

This flow will now show conversions only for events that resulted in a purchase as a result of the user having initially looked at a product that was recommended on the home page (user personalization) or as a result of a related product recommendation (similar products personalization).

![](images/amplitude/conversion-funnel-post-pers.png)

This funnel shows a combined conversion rate of 30.6%.  This means that for users that have clicked on products that are recommended by these two features, there is a 6.8% increase in conversion for all users that view recommended or similar products over users that find products organically.

Speaking of organic searches, let's how personalized search results ranking performed against regular searches.    

TODO:  Add all screen shots from workshop blog